In [119]:
import sys
import base64
import random
import chardet
import codecs
import json

import numpy as np

from re import sub
from collections import defaultdict

from collections import defaultdict, Counter

import pickle

from sklearn.feature_extraction import DictVectorizer

In [355]:
def read_x(path, dv, max_count = None):
    f = open(path, "r")
    
    p = f.readline().decode('utf-8')
    
    dict_list = []
    l_id = 0
    while p != '':
#         print l_id, len(dict_list)
        current = split.get_features(p, max_count)
        dict_list  += current
        
        p = f.readline().decode('utf-8')
        l_id += 1
    
    
    
    return dv.fit_transform(dict_list), dv, dv.vocabulary_
    

In [388]:
def read_y(path, max_count = None):
    f = open(path, "r")
    result = []
   
    
    p = f.readline()
    
    total_read = 0
    l_id = 0
    while p != '':
        p = json.loads(p)
        tmp = []
     
        for sentence in p["Sentences"]:
            n = sentence.count('.')
            if sentence[-1] == '.':
                n -= 1
            tmp += [0] * n
            if sentence[-1] == '.':
                tmp.append(1)
            total_read += n
        
            
            if max_count is not None and total_read >= max_count:
                return np.array(result)
        
#         if l_id == 77:
#             print tmp
#             for sentence in p["Sentences"]:
#                 print sentence
#                 n = sentence.count('.')
#                 print n
#                 print sentence[-1]
#                 if sentence[-1] == '.':
#                     n -= 1
# #                     tmp += [0] * n
#                 print n
                
            
#         print l_id
#         print tmp
#         print p["Paragraph"]
#         print l_id, len(result)
        result += tmp
        
        p = f.readline()
        l_id += 1
            
    
    return np.array(result)

In [401]:
import split
reload(split)

<module 'split' from 'split.py'>

In [402]:
x, dv, vocab = read_x("gold.txt", DictVectorizer(sparse=False), max_count=None)

In [403]:
from sklearn.tree import DecisionTreeClassifier

In [404]:
y = read_y("gold.json", max_count = None)

In [405]:
len(y)

380

In [406]:
x.shape

(380, 11)

In [407]:
x[:10]

array([[   0.,    0.,    0.,   62.,    1.,    0.,    0.,    0.,    0.,
           0.,    1.],
       [   0.,    0.,    1.,    9.,    1.,    9.,    0.,    0.,   60.,
           1.,    1.],
       [   0.,    0.,    7.,  104.,    1.,    1.,    0.,    0.,    7.,
           1.,    1.],
       [   0.,    0.,    6.,   10.,    1.,   13.,    0.,    0.,  102.,
           1.,    1.],
       [   0.,    0.,    1.,    3.,    0.,    3.,    0.,    0.,    8.,
           1.,    1.],
       [   0.,    0.,    1.,  108.,    0.,    1.,    0.,    0.,    1.,
           0.,    1.],
       [   0.,    0.,   10.,  108.,    1.,   12.,    0.,    0.,  106.,
           0.,    1.],
       [   0.,    1.,   13.,    1.,    0.,   11.,    0.,    0.,  106.,
           1.,    0.],
       [   0.,    0.,    0.,  160.,    0.,    0.,    0.,    0.,    0.,
           0.,    1.],
       [   0.,    0.,   13.,  113.,    1.,   22.,    0.,    0.,  158.,
           0.,    1.]])

In [408]:
from sklearn.cross_validation import cross_val_score 
from sklearn.ensemble import GradientBoostingClassifier

In [375]:
# model = DecisionTreeClassifier()
model = GradientBoostingClassifier(n_estimators=2500, subsample=0.7)

In [376]:
scores = cross_val_score(model, x, y, cv = 5, scoring='accuracy')

In [377]:
print scores
print scores.mean()

[ 0.93421053  0.94736842  0.93421053  0.96052632  0.96052632]
0.947368421053


In [378]:
model = model.fit(x, y)

In [379]:
pickle.dump(model, open("model.pickle", "w"))

In [380]:
pickle.dump(dv, open("vectorizer.pickle", "w"))

In [381]:
def print_importances(model, vocab):
    order = (-model.feature_importances_).argsort()
#     print vocab
    for i in order:
        print vocab.keys()[i], model.feature_importances_[i]

In [383]:
print_importances(model, vocab)

is_elipsis 0.90149463561
prev_starts_upper 0.0344367404271
is_end 0.0215026472968
next_is_upper= 0.0182089052459
len 0.0100111483177
last_word_len 0.00206570902096
prev_starts_upper= 0.0014700212981
prev_is_brace 0.00102075691841
prev_is_quote 0.00029823210718
next_is_upper 0.000291203757549
starts_with_space 0.0
prec_words 0.0
prev_len 0.0


In [350]:
# vocab